In [56]:
:opt no-lint

# 29 IO
## 29.1 IO
Topics:
- how `IO` works operationally
- what it should mean to you when you read a type that has `IO` in it
- more detail about the `IO` instances of `Functor`, `Applicative`, and `Monad`
## 29.2 Where IO explanations go astray
Some sources explain `IO` in terms of `State`. See:

In [1]:
:info IO

However, the `State` here is just a signalling mechanism for telling GHC the order your IO actions.
## 29.3 The reason we need this type
`IO` primarily exists to order operations and to disable some forms of sharing.
`IO` actions are instead enclosed within nested lambdas. Nesting is the only way to ensure that actions are sequenced within a pure lambda calculus.
`Monad` is a means of abstracting away the nested lambda noise that underlies `IO`.
## 29.4 Sharing
`IO` doesn’t disable all forms of sharing. Differently from other types, values of type `IO a` are not an `a`; they’re a description of how to get an `a`.
## 29.5 IO doesn’t disable sharing for everything
It only disables sharing for the terminal value it reduces to. Values that are not dependent on `IO` for their evaluation can still be shared.
## 29.6 Purity is losing meaning
- *Pure functional*: only lambda calculus semantics
- *Referential transparency*: same input implies the same output
A function that returns `IO a` is still referentially transparent, but executing many times the `IO a` may produce different results.
## 29.7 IO’s Functor, Applicative, and Monad
The `IO` Functor produces a new `IO` action in terms of the old one by transforming the final result of the old one.
The `IO` Applicative applies a lifted function to a lifted value.
With the `IO` Monad, moreover, the effects performed by the outer `IO` action can influence the inner part.
Associativity holds for the `IO` `>>=` operation.
## 29.8 Well, then, how do we MVar?
It is possible to break referential transparency with `unsafePerformIO`.
## 29.9 Chapter exercises
### File I/O with Vigenère


In [4]:
import           Data.Char
import           System.Environment (getArgs)
import           System.Exit
import           System.IO          (hGetLine, hPutStr, hWaitForInput, stderr,
                                     stdin, stdout)

charCodes :: [Int]
charCodes = fmap ord ['a'..'z']

wrapLeft :: Int -> Int
wrapLeft n
    | n > (last charCodes) = n - (last charCodes) + (head charCodes - 1)
    | otherwise            = n

wrapRight :: Int -> Int
wrapRight n
    | n < (head charCodes) = n + (last charCodes) - (head charCodes - 1)
    | otherwise            = n

shiftChar :: Int -> Char -> Char
shiftChar n ch = chr $ wrapLeft (n + ord ch)

unshiftChar :: Int -> Char -> Char
unshiftChar n co = chr $ wrapRight (ord co - n)

caesar :: Int -> String -> String
caesar _ [] = []
caesar n xs = fmap (shiftChar n) xs

uncaesar :: Int -> String -> String
uncaesar _ [] = []
uncaesar n xs = fmap (unshiftChar n) xs

table :: [(Char, Int)]
table = zip ['a'..'z'] [0..]

readIn :: String -> Int
readIn x = read x

parseArgs :: Int -> String -> String -> String
parseArgs k "-d" xs = uncaesar k xs
parseArgs k _    xs = caesar k xs

finishWithError :: IO a
finishWithError = do
  hPutStr stderr "Timed out"
  exitWith (ExitFailure 1)

run :: IO ()
run = do
  (k:m:[]) <- getArgs
  putStrLn "Start typing."
  success <- hWaitForInput stdin (5000)
  input <- case success of
    False -> finishWithError
    True  -> hGetLine stdin
  putStrLn $ parseArgs (readIn k) m input
  
run

Line 12: Redundant bracket
Found:
n > (last charCodes)
Why not:
n > last charCodesLine 12: Redundant bracket
Found:
n - (last charCodes)
Why not:
n - last charCodesLine 17: Redundant bracket
Found:
n < (head charCodes)
Why not:
n < head charCodesLine 17: Redundant bracket
Found:
n + (last charCodes)
Why not:
n + last charCodesLine 38: Eta reduce
Found:
readIn x = read x
Why not:
readIn = readLine 51: Use list literal pattern
Found:
(k : m : [])
Why not:
[k, m]Line 53: Redundant bracket
Found:
(5000)
Why not:
5000Line 54: Use if
Found:
case success of
  False -> finishWithError
  True -> hGetLine stdin
Why not:
if success then hGetLine stdin else finishWithErrorLine 56: Use getLine
Found:
hGetLine stdin
Why not:
getLine

: 

### Config directories

In [5]:
import           Control.Applicative
import           Data.ByteString     (ByteString)
import           Data.Char           (isAlpha)
import           Data.Map            (Map)
import qualified Data.Map            as M
import           Data.Text           (Text)
import qualified Data.Text.IO        as TIO
import           Text.RawString.QQ
import           Text.Trifecta

headerEx :: ByteString
headerEx = "[blah]"

newtype Header = Header String
  deriving (Eq, Ord, Show)

parseBracketPair :: Parser a -> Parser a
parseBracketPair p = char '[' *> p <* char ']'

parseHeader :: Parser Header
parseHeader = parseBracketPair (Header <$> some letter)

assignmentEx :: ByteString
assignmentEx = "woot=1"

type Name        = String
type Value       = String
type Assignments = Map Name Value

parseAssignment :: Parser (Name, Value)
parseAssignment = do
  name <- some letter
  _    <- char '='
  val  <- some (noneOf "\n")
  skipEOL
  return (name, val)

skipEOL :: Parser ()
skipEOL = skipMany (oneOf "\n")

commentEx :: ByteString
commentEx = "; last modified April\
            \ 2001 by John Doe"

commentEx' :: ByteString
commentEx' = "; blah\n; woot\n \n;hah"

skipComments :: Parser ()
skipComments = skipMany $ do
  _ <- char ';' <|> char '#'
  skipMany (noneOf "\n")
  skipEOL

sectionEx :: ByteString
sectionEx = "; ignore me\n[states]\nChris=Texas"

sectionEx' :: ByteString
sectionEx' = [r|
; ignore me
[states]
Chris=Texas
|]

sectionEx'' :: ByteString
sectionEx'' = [r|
;comment
[section]
host=wikipedia.org
alias=claw

[whatisit]
red=intoothandclaw
|]

data Section = Section Header Assignments
  deriving (Eq, Show)

newtype Config = Config (Map Header Assignments)
  deriving (Eq, Show)

skipWhitespace :: Parser ()
skipWhitespace = skipMany (char ' ' <|> char '\n')

parseSection :: Parser Section
parseSection = do
  skipWhitespace
  skipComments
  h <- parseHeader
  skipEOL
  assignments <- some parseAssignment
  return $ Section h (M.fromList assignments)

rollup :: Section -> Map Header Assignments -> Map Header Assignments
rollup (Section h a) m = M.insert h a m

parseIni :: Parser Config
parseIni = do
  sections <- some parseSection
  let mapOfSections = foldr rollup M.empty sections
  return (Config mapOfSections)

maybeSuccess :: Result a -> Maybe a
maybeSuccess (Success a) = Just a
maybeSuccess _           = Nothing

run :: IO ()
run = print $ parseByteString parseAssignment mempty assignmentEx

: 

: 

: 

: 

In [6]:
import           Data.List          (isSuffixOf)
import qualified Data.Map           as M
import           Parser
import           System.Directory
import           System.Environment

type Entry = M.Map FilePath String

selectConfigs :: [String] -> [String]
selectConfigs = filter (isSuffixOf ".ini")

readC :: FilePath -> IO Entry -> IO Entry
readC x iomp = do
  v <- readFile x
  mp <- iomp
  return $ M.insert x v mp

ls :: IO ()
ls = do
  [x] <- getArgs
  paths <- listDirectory x
  files <- return $ selectConfigs paths
  mp <- foldr readC (return M.empty) files
  print mp

Line 13: Use <$>
Found:
do v <- readFile x
   mp <- iomp
   return $ M.insert x v mp
Why not:
do v <- readFile x
   M.insert x v <$> iompLine 19: Use let
Found:
do [x] <- getArgs
   paths <- listDirectory x
   files <- return $ selectConfigs paths
   mp <- foldr readC (return M.empty) files
   print mp
Why not:
do [x] <- getArgs
   paths <- listDirectory x
   let files = selectConfigs paths
   mp <- foldr readC (return M.empty) files
   print mp

: 